In [1]:
from bs4 import BeautifulSoup
import pandas as pd
import requests
from datetime import date
from datetime import timedelta
from datetime import datetime
from tqdm import tqdm
from tqdm.notebook import tqdm_notebook
from dotenv import load_dotenv
import hashlib

tqdm_notebook.pandas()

load_dotenv('../secrets/.env', override=True)

%run web_scraping.ipynb
%run useful_functions.ipynb

/opt/anaconda3/lib/python3.8/site-packages/tqdm/std.py:697: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


In [2]:
shows = get_shows()
show_broadcast_details = get_show_broadcast_details()
broadcast_details = get_broadcast_details()
broadcast_playlists = get_broadcast_playlists()

In [3]:
#shows_to_run = ["https://www.bbc.co.uk/programmes/m0015f4k/episodes/guide",
#                "https://www.bbc.co.uk/programmes/b09c19f4/episodes/guide",
#                "https://www.bbc.co.uk/programmes/m000zd5w/episodes/guide",
#                "https://www.bbc.co.uk/programmes/b00rz79k/episodes/guide",
#                "https://www.bbc.co.uk/programmes/b00t3nsw/episodes/guide",
#                "https://www.bbc.co.uk/programmes/b006wq4s/episodes/guide",
#                "https://www.bbc.co.uk/programmes/m001bhnd/episodes/guide",
#                "https://www.bbc.co.uk/programmes/m001bjdj/episodes/guide",
#                "https://www.bbc.co.uk/programmes/m0000blt/episodes/guide",
#                "https://www.bbc.co.uk/programmes/m00008zy/episodes/guide", 
#                "https://www.bbc.co.uk/programmes/b03hjfww/episodes/guide",
#                "https://www.bbc.co.uk/programmes/b006wkh9/episodes/guide",
#                "https://www.bbc.co.uk/programmes/p04m6srg/episodes/guide",
#                "https://www.bbc.co.uk/programmes/m000m5bw/episodes/guide",
#                "https://www.bbc.co.uk/programmes/m0016zh5/episodes/guide",
#                "https://www.bbc.co.uk/programmes/b0072lb2/episodes/guide",
#                "https://www.bbc.co.uk/programmes/b0072q60/episodes/guide",
#                "https://www.bbc.co.uk/programmes/b00p4l4t/episodes/guide",
#                "https://www.bbc.co.uk/programmes/b0072l4x/episodes/guide"]

temp_show_broadcast_details = pd.DataFrame()

#for show_url in tqdm(shows_to_run):
for show_url in tqdm(shows['Show URL'], desc = 'Show'):
    show_instance = Show(show_url, max_pages = 2)
    if shows.empty or show_instance.show_key not in shows['Show Key'].unique():
        shows = pd.concat([shows,show_instance.show_details])
    temp_show_broadcast_details = pd.concat([temp_show_broadcast_details,show_instance.broadcasts])
    
temp_show_broadcast_details['Datetime Key'] = datetime.now()
temp_show_broadcast_details['Is New Data'] = True
temp_show_broadcast_details['Is Maybe Pending'] = True
temp_show_broadcast_details.drop(columns=['Broadcast Number'], inplace=True)

show_broadcast_details = pd.concat([show_broadcast_details,temp_show_broadcast_details])
show_broadcast_details = show_broadcast_details.groupby(allbut(show_broadcast_details.columns,['Datetime Key','Is New Data','Is Maybe Pending'])).apply(lambda group: group.nsmallest(1, columns='Datetime Key')).reset_index(drop=True)
show_broadcast_details = show_broadcast_details.groupby("Broadcast Key").apply(lambda group: group.nlargest(1, columns='Datetime Key')).reset_index(drop=True)

del(temp_show_broadcast_details, 
    show_instance, 
    show_url)

try:
    del(shows_to_run)
except:
    pass

Show: 100%|██████████| 19/19 [00:14<00:00,  1.31it/s]


In [4]:
max_trials = 1000
#new_broadcast_data = show_broadcast_details[(show_broadcast_details["Is Broadcast Upcoming"]==False) & ((show_broadcast_details["Is New Data"]==True) | (show_broadcast_details['Is Maybe Pending'] == True))][0:max_trials]
new_broadcast_data = show_broadcast_details[((show_broadcast_details["Is New Data"]==True) | (show_broadcast_details['Is Maybe Pending'] == True))][0:max_trials]
temp_broadcast_details = pd.DataFrame()
temp_broadcast_playlists = pd.DataFrame()
run_keys = []

for i, [broadcast_url,broadcast_key] in enumerate(tqdm(new_broadcast_data[['Broadcast URL','Broadcast Key']].values, desc = 'Broadcast')):
    try:
        broadcast_instance = Playlist(broadcast_url)
        if ((len(broadcast_instance.playlist.index) == 0) & ((pd.to_datetime('now')- broadcast_instance.broadcast_details["Service Date"]).dt.days.values <= 7)) | ((pd.to_datetime('now')- broadcast_instance.broadcast_details["Service Date"]).dt.days.values == 0):
            show_broadcast_details.loc[show_broadcast_details["Broadcast Key"] == broadcast_key,'Is Maybe Pending'] = True
        else:
            show_broadcast_details.loc[show_broadcast_details["Broadcast Key"] == broadcast_key,'Is Maybe Pending'] = False
        temp_broadcast_details = pd.concat([temp_broadcast_details,broadcast_instance.broadcast_details])
        temp_broadcast_playlists = pd.concat([temp_broadcast_playlists,broadcast_instance.playlist])
        run_keys.append(broadcast_key)
    except:
        pass
    
temp_broadcast_details['Datetime Key'] = datetime.now()
temp_broadcast_details['Is New Data'] = True
temp_broadcast_playlists['Datetime Key'] = datetime.now()
temp_broadcast_playlists['Is New Data'] = True

if broadcast_details.empty:
    of_interest_details_mask = None
    broadcast_details_update = broadcast_details
    broadcast_details_stationary = broadcast_details
else:
    of_interest_details_mask = broadcast_details['Broadcast Key'].apply(lambda x: x  in list(temp_broadcast_details['Broadcast Key']))
    broadcast_details_update = broadcast_details[of_interest_details_mask]
    broadcast_details_stationary = broadcast_details[~of_interest_details_mask]
broadcast_details_update = pd.concat([broadcast_details_update,temp_broadcast_details])
broadcast_details_update = broadcast_details_update.groupby(allbut(broadcast_details_update.columns,['Datetime Key','Is New Data'])).apply(lambda group: group.nsmallest(1, columns='Datetime Key')).reset_index(drop=True)
broadcast_details_update = broadcast_details_update.groupby("Broadcast Key").apply(lambda group: group.nlargest(1, columns='Datetime Key')).reset_index(drop=True)
broadcast_details_update['Is New Data'] = False
broadcast_details = pd.concat([broadcast_details_update,broadcast_details_stationary])

if broadcast_playlists.empty:
    of_interest_playlist_mask = None
    broadcast_playlists_update = broadcast_playlists
    broadcast_playlists_stationary = broadcast_playlists
else:
    of_interest_playlist_mask = broadcast_playlists['Broadcast Key'].apply(lambda x: x  in list(temp_broadcast_playlists['Broadcast Key']))
    broadcast_playlists_update = broadcast_playlists[of_interest_playlist_mask]
    broadcast_playlists_stationary = broadcast_playlists[~of_interest_playlist_mask]
broadcast_playlists_update = pd.concat([broadcast_playlists_update,temp_broadcast_playlists])
broadcast_playlists_update = broadcast_playlists_update.groupby(allbut(broadcast_playlists_update.columns,['Datetime Key','Is New Data'])).apply(lambda group: group.nsmallest(1, columns='Datetime Key')).reset_index(drop=True)
broadcast_playlists_update = broadcast_playlists_update.groupby(["Broadcast Key", "Broadcast Position"]).apply(lambda group: group.nlargest(1, columns='Datetime Key')).reset_index(drop=True)
broadcast_playlists_update['Is New Data'] = False
broadcast_playlists = pd.concat([broadcast_playlists_update,broadcast_playlists_stationary])

show_broadcast_details['Is New Data'] = show_broadcast_details.apply(lambda x: False if ((x['Is New Data'] == False) | (x['Broadcast Key'] in run_keys)) else True, axis = 1)
                         
del(i,
    max_trials,
    broadcast_url,
    broadcast_key,
    run_keys,
    new_broadcast_data,
    broadcast_instance,
    temp_broadcast_playlists,
    temp_broadcast_details,
    broadcast_playlists_update,
    broadcast_playlists_stationary,
    of_interest_details_mask,
    of_interest_playlist_mask,
    broadcast_details_update,
    broadcast_details_stationary)


Broadcast: 100%|██████████| 141/141 [00:57<00:00,  2.46it/s]


In [5]:
shows.to_pickle("../data/shows.pkl")
show_broadcast_details.to_pickle("../data/show_broadcast_details.pkl")
broadcast_details.to_pickle("../data/broadcast_details.pkl")
broadcast_playlists.to_pickle("../data/broadcast_playlists.pkl")